In [1]:
import time
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets
from IPython.display import display
import random

In [2]:
class EXP_Output:
    def __init__(self):
        #self.out = widgets.Output(layout={'border': '1px solid black'})
        #display(self.out)
        #with self.out:
        #print("Experiment outputs")

        self.Queue_Print = Queue()
        self.thread = Thread(target=self.loop_Print)
        self.thread.start()

    def print(self, msg):
        text_out = time.strftime("%H:%M:%S", time.localtime()) + "> "
        self.Queue_Print.put(text_out+msg)

    def loop_Print(self):
        while True:
            msg = self.Queue_Print.get()
            #with self.out:
            print(msg)
            self.Queue_Print.task_done()

In [3]:
class ALE_TextInput:
    
    def __init__(self, exp_out):
        
        self.Queue_User = Queue()
        self.exp_out = exp_out

        # Create a text input widget
        self.Text = widgets.Text(
                        value='',
                        placeholder='Type something and press enter',
                        description='To send:',
                        disabled=False
                    )

        # create a button object
        self.Botton =  widgets.Button(
                            description='Submit',
                            disabled=False,
                            button_style='', 
                            tooltip='Click me to submit a string',
                        )

        # Define a function to handle the click action on the button
        def on_submit_button_clicked(b):

            global thread_running
            
            msg = self.Text.value
            #self.exp_out.print("ALE_0: received input: "+str(msg))
                
            if thread_running == True:
                if msg == "end":
                    thread_running = False
                
                for i in range(20):
                    self.Queue_User.put(msg+str(i))

            self.Text.value = ''  # Clear the input field after submission

        # Attach the event to the button
        self.Botton.on_click(on_submit_button_clicked)
        
        # Display the widgets
        display(self.Text, self.Botton)
        
    # to get a message from user queue. this function can block the thread
    def get(self):
        return self.Queue_User.get()

In [4]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR, exp_out):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx  # must provide get()
        self.Lower_TR = lower_TR  # must provide send(msg), and receive()
        self.exp_out  = exp_out
        
    def loop_Tx(self):

        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):

            c = c + 1

            # get text from Upper_Tx, which must provide a get method
            # this thread is blocked here
            msg = self.Upper_Tx.get()
            
            text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

            # add the text to the queue
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)

In [5]:
# state of DLE
STATE_READY_TO_SEND = 0 # ready to send a packet to the DLE entity in another node through lower layer
STATE_WAITING_ACK   = 1 # waiting for ACK from the DLE entity in another node

# events of DLE
EVENT_UPPER_TX  = 0 # upper layer wants to send a packet
EVENT_LOWER_DAT = 1 # lower layer forwards an incoming data packet
EVENT_LOWER_ACK = 2 # lower layer forwards an incoming data packet
EVENT_TIMEOUT   = 3 # timer timeout
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR, exp_out):
        
        self.Name      = name
        self.Lower_TR  = lower_TR   # must provide send(msg) and receive()
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
        # create a queue for Finite State Machine
        self.Queue_FSM = Queue()
        
        # create a flag = condition to start sending
        self.cv_TxOp = Condition()
        self.TxOp    = True

        # init state to STATE_WAITING_UPPER
        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]] 
        

        # new features for retransmission
        # timer
        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        # transmission 
        self.tx_buffer  = ''
        self.tx_seq_num = '0'
        
        # reception
        self.tx_ack_num = '0'
        
        
    def loop_Tx(self):

        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):

            #Delay loop_Tx for a range of 0 to 2 seconds
            delay = random.randrange(0,2)
            time.sleep(delay)

            # waiting for the ready to sent signal
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                
                c = c + 1

                # get a message from queue
                msg = self.Queue_Tx.get()
                text_out = self.Name + " Tx: message " + str(c) + ": " + str(msg)
                self.exp_out.print(text_out)
            
                self.event_add(EVENT_UPPER_TX, msg) 

    def loop_Rx(self):
        
        global thread_running
        c = 0
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            
            text_out = self.Name + " Rx: message " + str(c) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            # add an event for FSM, assuming that msg is a sequence of bytes
            msg_type = msg[0]
            if (msg_type == '0'):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == '1'):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                text_out = self.Name + " Rx: message type unknown " + msg_type
                self.exp_out.print(text_out)

    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        
        # preparing an event
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        # the event is a sequence of bytes
        event = ev_type.to_bytes(1, "big")+msg
        
        # add event to queue
        self.Queue_FSM.put(event)        
        

    def loop_FSM(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_FSM starting")
        
        while (thread_running == True):
            
            # get the next event
            event = self.Queue_FSM.get()
            
            ev_type = event[0]
            
            text_out = self.Name + " FSM: state: " + str(self.State)+" event type: " + str(ev_type)
            self.exp_out.print(text_out)
            
            # process the event
            msg = event[1:].decode('utf-8') # message becomes a string
            self.Procedure[ev_type][self.State](msg)
        
    def FSM_abnormal(self, msg):
        
        text_out = self.Name + " FSM: error! " + msg
        self.exp_out.print(text_out)
    
    def FSM_upper_Tx(self, msg):   
        
        text_out = self.Name + " FSM: to send: " + msg
        self.exp_out.print(text_out)

        # buffer the message (for retransmission)
        self.tx_buffer = msg
        
        # prepare to send a data message
        msg = '0' + self.tx_seq_num + msg     # inicating a new packet and its sequence number
        self.Lower_TR.send(msg)

        # waiting for ACK
        self.State = STATE_WAITING_ACK
        
        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()    
    
    def FSM_lower_Rx(self, msg):

        # get the sequence number
        seq = msg[0]
        msg = msg[1:]
        
        # received a new data packet        
        text_out = self.Name + " FSM: received: " + msg
        self.exp_out.print(text_out)

        if (seq == self.tx_ack_num):
            
            # put the message in a receiving queue
            self.Queue_Rx.put(msg)
            
            self.tx_ack_num = chr(ord(self.tx_ack_num) + 1)
            if (self.tx_ack_num == chr(ord('9') + 1)):
                self.tx_ack_num = '0'
        
        else:
            text_out  = self.Name + " FSM: received received frame " + seq
            text_out += " but expected " + self.tx_ack_num
            self.exp_out.print(text_out)
            
            # to send an ACK
            self.Lower_TR.send('1'+seq)

                
    def FSM_lower_Rx_ack(self, msg):
        
        ack = msg[0] # get the ack number
        if (ack != self.tx_seq_num):
            
            # the received ack does not match the seq
            text_out = "ACK # " + ack + " does not match local seq # " + self.tx_seq_num
            self.exp_out.print(text_out)
            return

        # stop timer
        self.timer_active = False
        
        # inc the seq
        self.tx_seq_num = chr(ord(self.tx_seq_num) + 1)
        if (self.tx_seq_num == chr(ord('9') + 1)):
                self.tx_seq_num = '0'
        
        # ready to send the next upper layer packet
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        
        text_out = self.Name + " FSM: to resend frame " + self.tx_seq_num + " " + self.tx_buffer
        self.exp_out.print(text_out)
        
        # prepare to send a data message
        msg = '0' + self.tx_seq_num + self.tx_buffer     # inicating a new packet and its sequence number
        self.Lower_TR.send(msg)
        
        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_Timer starting")
        
        while (thread_running == True):
            
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
                time.sleep(0.5)

                if (self.timer_counter == 0):
                    # add an event for timeout
                    self.event_add(EVENT_TIMEOUT, 'x')
                    self.timer_active = False

                else:
                    self.timer_counter = self.timer_counter - 1

In [6]:
class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx, exp_out):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        self.exp_out   = exp_out
        
    def loop_Tx(self):
        
        global thread_running
        self.exp_out.print(self.Name + ": loop_Tx starting")
    
        while (thread_running == True):
        
            # get a message from queue
            msg = self.Queue_Tx.get()
            
            text_out = self.Name + " Tx: message: " + str(msg)
            self.exp_out.print(text_out)
            
            time.sleep(0.5)
            
            # sending the message using socket
            msg_bytes = str.encode(msg)
            if (random.random() < 0.1):#message fails and we send a packet loss error message
                self.exp_out.print("packet loss")
            else:
                # sending the message using socket
                self.Socket.sendto(msg_bytes, self.AP_Tx)
                if(random.random() <= 0.05):
                    self.Socket.sendto(msg_bytes, self.AP_Tx) #Message will get duplicated.

    def loop_Rx(self):
        
        global thread_running
        global bufferSize
        self.exp_out.print(self.Name + ": loop_Rx starting")
    
        # binding the socket with the IP and port
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
        
            # get a message from socket, this thread is blocked here
            msg_addr = self.Socket.recvfrom(bufferSize)
    
            msg  = msg_addr[0].decode('utf-8')
            addr = msg_addr[1]
            
            text_out = self.Name + " Rx: from " + str(addr) + ": " + str(msg)
            self.exp_out.print(text_out)
            
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [7]:
thread_running = False
bufferSize = 1024

# (0) creating an output object
exp_out = EXP_Output()

# (1) create physical layer entities
AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1, exp_out)

# (2) create date link layer entities
DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1, exp_out)

# (3) create application layer entities
ALE_0 = ALE_TextInput(exp_out)
ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

17:52:18> ALE_Alice: loop_Tx starting
17:52:18> ALE_Alice: loop_Rx starting
17:52:18> DLE_Alice: loop_Tx starting
17:52:18> DLE_Alice: loop_Rx starting
17:52:18> PLE_Alice: loop_Tx starting
17:52:18> PLE_Alice: loop_Rx starting
17:52:18> ALE_Bob: loop_Tx starting
17:52:18> ALE_Bob: loop_Rx starting
17:52:18> DLE_Bob: loop_Tx starting
17:52:18> DLE_Bob: loop_Rx starting
17:52:18> PLE_Bob: loop_Tx starting
17:52:18> PLE_Bob: loop_Rx starting
17:52:18> DLE_Alice: loop_FSM starting
17:52:18> DLE_Bob: loop_FSM starting
17:52:18> DLE_Alice: loop_Timer starting
17:52:18> DLE_Bob: loop_Timer starting


In [8]:
# (4) create physical layer entities
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2, exp_out)

# (5) create date link layer entities
DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2, exp_out)

# (6) create application layer entities
ALE_3 = ALE_TextInput(exp_out)
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)

Text(value='', description='To send:', placeholder='Type something and press enter')

Button(description='Submit', style=ButtonStyle(), tooltip='Click me to submit a string')

In [9]:
# start the loops of all entities
# all loops must be blocked at a certain position

t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())


t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ()) 
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())

thread_running = True

t1_1.start()
t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

Exception in thread Thread-11 (loop_Rx):
Traceback (most recent call last):
  File "C:\Users\ashth\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\ashth\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
Exception in thread Thread-19 (loop_Rx):
Traceback (most recent call last):
  File "C:\Users\ashth\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    _threading_Thread_run(self)
  File "C:\Users\ashth\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self.run()
  File "C:\Users\ashth\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ashth\AppData\Local\Temp\ipykernel_8872\2856713484.py", line 45, in loop_Rx
    _threading_Thread_run(self)
  File "C:\Users\ashth\AppDat

In [10]:
#New Measurement Layer
class MLE_TR:
    def __init__(self, name, lower_TR, exp_out):
        #New button to test the MLE
        self.button = widgets.Button(description="Measurement",disabled=False)

        self.name = name
        self.exp_out = exp_out

        # must provide a send() and receive() interface
        self.lower_TR = lower_TR
        self.queue_Rx = Queue()
        self.queue_Tx = Queue()
        self.dupes = set() #A set to keep track of duplicates

        
        def on_button_clicked(b):

            #Send a 100 messages these need to indicate experiment id, message id and current time up to a millisecond
            global thread_running    
            if thread_running == True:
                for i in range(1,101):
                    msg = str("Measurements:(%d,%d,%d)" % (i, i, time.time_ns()/1000000))
                    self.lower_TR.send(msg) #incase


            #self.exp_out.print("Button has been clicked.")

        # bind the callback function to the button
        # and display the button
        self.button.on_click(on_button_clicked)
        display(self.button)

#Message from Lower Layer
    def send(self, msg):
        self.queue_Rx.put(msg)
        
#Message from transmitter
    def get(self):
        return self.queue_Tx.get()

    def loop_Rx(self):
        global thread_running
        global buffer_size
        #Keep count of the packets sent to have more statistics
        cnt = 0
        dupes_sum = 0
        avg_time_sum = 0
        
        self.exp_out.print("MLE_TR method: loop_RX is active.")
        
        

        while (thread_running == True):
            self.exp_out.print("inner loop is working.")
            cnt += 1
            # get a message from the lower layer
            msg = self.lower_TR.receive()
            
            
            #Extract information
            if "Measurements:" in msg:
                # Extract data
                # Start by getting what is between the parentheses
                start = msg.find("(")
                end = msg.find(")")
                data = msg[start + 1:end]

                # Split the data
                data = data.split(",")

                # Convert the data to integers
                #data = [int(i) for i in data]

                #self.buttons[data[1]] += 1
                #self.delay_sum += (time.time_ns() - data[2])


                if data[1] in self.dupes: #If message is in dupes
                    dupes_sum += 1
                
                else:
                    self.dupes.add(int(data[1]))

                lost_packets = 100 - (cnt - dupes_sum)
                avg_time_sum = avg_time_sum + round(time.time_ns()/1000000) -int(data[2])
    
                # Print the data
                #print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
                #print("%s: Rx Button: %s" % (self.name, msg))
                self.exp_out.print("\t* Experiment ID: %d" % int(data[0]))
                self.exp_out.print("\t* Message ID: %d" % int(data[1]))
                self.exp_out.print("\t* Time: %d" % int(data[2]))
                self.exp_out.print("\t* Avg. Message Loss Rate: " + str(lost_packets/100)) 
                self.exp_out.print("\t* Avg. Time Taken: " + str(avg_time_sum/cnt))
                self.exp_out.print("\t* Avg. Duped Messages: " + str(dupes_sum/100))
                #print("\t* Delay: %d" % (time.time_ns() - data[2]))

                # Send an acknowledgement for the other Measurement Layer
                #ack = str("Button Ack:(%d,%d,%d,%d)" % (data[0], data[1], time.time_ns(), time.time_ns() - data[2]))
                #self.lower_TR.send(ack)


In [11]:
#Experiment 3
thread_running = False
bufferSize = 1024

# (0) creating an output object
exp_out1 = EXP_Output()

# (1) create physical layer entities
AP_local_3  = ("127.0.0.1", 40000)
AP_remote_3 = ("127.0.0.1", 41111)
Socket_3    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_3       = PLE_TR("PLE_Alice3", Socket_3, AP_remote_3, AP_local_3, exp_out1)

# (2) create date link layer entities
DLE_3 = DLE_TR_FSM("DLE_Alice3", PLE_3, exp_out1)
MLE_1 = MLE_TR("MLE_Alice3", DLE_3, exp_out1)#MLE_TR object is the upper layer of PLE_TR object

# (3) create application layer entities
#ALE_0 = ALE_TextInput(exp_out)
#ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1, exp_out)

Button(description='Measurement', style=ButtonStyle())

17:52:18> DLE_Alice3: loop_Tx starting
17:52:18> DLE_Alice3: loop_Rx starting
17:52:19> PLE_Alice3: loop_Tx starting
17:52:19> PLE_Alice3: loop_Rx starting
17:52:19> DLE_Bob4: loop_Tx starting
17:52:19> DLE_Bob4: loop_Rx starting
17:52:19> PLE_Bob4: loop_Tx starting
17:52:19> PLE_Bob4: loop_Rx starting
17:52:19> DLE_Alice3: loop_FSM starting
17:52:19> DLE_Bob4: loop_FSM starting
17:52:19> DLE_Alice3: loop_Timer starting
17:52:19> DLE_Bob4: loop_Timer starting
17:52:19> MLE_TR method: loop_RX is active.
17:52:19> inner loop is working.
17:52:19> MLE_TR method: loop_RX is active.
17:52:19> inner loop is working.
18:34:42> DLE_Alice3 Tx: message 1: Measurements:(1,1,1712097282726)
18:34:42> DLE_Alice3 FSM: state: 0 event type: 0
18:34:42> DLE_Alice3 FSM: to send: Measurements:(1,1,1712097282726)
18:34:42> PLE_Alice3 Tx: message: 00Measurements:(1,1,1712097282726)
18:34:43> packet loss
18:34:48> DLE_Alice3 FSM: state: 1 event type: 3
18:34:48> DLE_Alice3 FSM: to resend frame 0 Measurements

In [12]:
# (4) create physical layer entities
AP_local_4  = ("127.0.0.1", 41111)
AP_remote_4 = ("127.0.0.1", 40000)
Socket_4    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_4       = PLE_TR("PLE_Bob4", Socket_4, AP_remote_4, AP_local_4, exp_out1)

# (5) create date link layer entities
DLE_4 = DLE_TR_FSM("DLE_Bob4", PLE_4, exp_out1)
MLE_2 = MLE_TR("MLE_Bob", DLE_4, exp_out1)#MLE_TR object is the upper layer of PLE_TR object

# (6) create application layer entities
#ALE_3 = ALE_TextInput(exp_out)
#ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2, exp_out)

Button(description='Measurement', style=ButtonStyle())

In [13]:
# start the loops of all entities
# all loops must be blocked at a certain position

t9_1 = Thread(target = DLE_3.loop_Tx, args = ())
t10_1 = Thread(target = DLE_3.loop_Rx, args = ())
t11_1 = Thread(target = PLE_3.loop_Tx, args = ()) 
t12_1 = Thread(target = PLE_3.loop_Rx, args = ())
f3_1 = Thread(target = DLE_3.loop_FSM, args = ())
f4_1 = Thread(target = DLE_3.loop_timer, args = ())
f5_1 = Thread(target=MLE_1.loop_Rx, args=())
f5_2 = Thread(target=MLE_2.loop_Rx, args=())


t9_2 = Thread(target = DLE_4.loop_Tx, args = ())
t10_2 = Thread(target = DLE_4.loop_Rx, args = ())
t11_2 = Thread(target = PLE_4.loop_Tx, args = ()) 
t12_2 = Thread(target = PLE_4.loop_Rx, args = ()) 
f3_2 = Thread(target = DLE_4.loop_FSM, args = ())
f4_2 = Thread(target = DLE_4.loop_timer, args = ())
thread_running = True


t9_1.start()
t10_1.start()
t11_1.start()
t12_1.start()
t9_2.start()
t10_2.start()
t11_2.start()
t12_2.start()
f3_1.start()
f3_2.start()
f4_1.start()
f4_2.start()
f5_1.start()
f5_2.start()

In [14]:
#Experiment 2 layer
thread_running = False
bufferSize = 1024

# (0) creating an output object
exp_out2 = EXP_Output()

# (1) create physical layer entities
AP_local_5  = ("127.0.0.1", 50000)
AP_remote_5 = ("127.0.0.1", 51111)
Socket_5    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_5       = PLE_TR("PLE_Alice3", Socket_5, AP_remote_5, AP_local_5, exp_out2)

# (2) create Measure layer entities
MLE_3 = MLE_TR("MLE_Alice", PLE_5, exp_out2)

Button(description='Measurement', style=ButtonStyle())

17:52:20> PLE_Alice3: loop_Tx starting
17:52:20> PLE_Alice3: loop_Rx starting
17:52:20> PLE_Bob4: loop_Tx starting
17:52:20> PLE_Bob4: loop_Rx starting
17:52:20> MLE_TR method: loop_RX is active.
17:52:20> inner loop is working.
17:52:20> MLE_TR method: loop_RX is active.
17:52:20> inner loop is working.
18:33:16> PLE_Alice3 Tx: message: Measurements:(1,1,1712097196920)
18:33:17> PLE_Alice3 Tx: message: Measurements:(2,2,1712097196921)
18:33:17> PLE_Bob4 Rx: from ('127.0.0.1', 50000): Measurements:(1,1,1712097196920)
18:33:17> 	* Experiment ID: 1
18:33:17> 	* Message ID: 1
18:33:17> 	* Time: 1712097196920
18:33:17> 	* Avg. Message Loss Rate: 0.99
18:33:17> 	* Avg. Time Taken: 510.0
18:33:17> 	* Avg. Duped Messages: 0.0
18:33:17> inner loop is working.
18:33:17> PLE_Alice3 Tx: message: Measurements:(3,3,1712097196921)
18:33:17> PLE_Bob4 Rx: from ('127.0.0.1', 50000): Measurements:(2,2,1712097196921)
18:33:17> 	* Experiment ID: 2
18:33:17> 	* Message ID: 2
18:33:17> 	* Time: 171209719692

In [15]:
# (3) create physical layer entities
AP_local_6  = ("127.0.0.1", 51111)
AP_remote_6 = ("127.0.0.1", 50000)
Socket_6    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_6       = PLE_TR("PLE_Bob4", Socket_6, AP_remote_6, AP_local_6, exp_out2)

# (4) create application layer entities
MLE_4 = MLE_TR("MLE_Bob", PLE_6, exp_out2)

Button(description='Measurement', style=ButtonStyle())

In [16]:
# start the loops of all entities
# all loops must be blocked at a certain position

t13_1 = Thread(target = PLE_5.loop_Tx, args = ()) 
t14_1 = Thread(target = PLE_5.loop_Rx, args = ())
f6_1 = Thread(target=MLE_3.loop_Rx, args=())



t13_2 = Thread(target = PLE_6.loop_Tx, args = ()) 
t14_2 = Thread(target = PLE_6.loop_Rx, args = ())
f6_2 = Thread(target=MLE_4.loop_Rx, args=()) 
thread_running = True



t13_1.start()
t14_1.start()
t13_2.start()
t14_2.start()
f6_1.start()
f6_2.start()